In [ ]:
import nglview
from openff.toolkit import ForceField, Molecule
from openff.units import Quantity, unit

from openff.interchange.interop._virtual_sites import get_positions_with_virtual_sites

sage_with_example_virtual_sites = ForceField(
    "openff-2.1.0.offxml",
    "example-vsites-parameters-forcefield.offxml",
)
sage_with_example_virtual_sites["VirtualSites"].parameters[-1].distance *= 0.1

sage_with_example_virtual_sites["VirtualSites"].parameters[-1].distance

In [ ]:
def run(smiles: str, force_field=sage_with_example_virtual_sites) -> nglview.NGLWidget:
    import openmm
    import openmm.unit

    molecule = Molecule.from_smiles(smiles, allow_undefined_stereo=True)

    if smiles == "N":
        molecule._conformers = [
            unit.Quantity(
                [
                    [0, 0, 1],
                    [1, 0, 0],
                    [-1, -1, 0],
                    [-1, 1, 0],
                ],
                unit.angstrom,
            )
        ]
    else:
        molecule.generate_conformers(n_conformers=1)

    interchange = force_field.create_interchange(molecule.to_topology())

    return interchange._visualize_nglview(include_virtual_sites=True)
    integrator = openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    simulation = interchange.to_openmm_simulation(integrator=integrator)

    # simulation.context.setPositions(get_positions_with_virtual_sites(interchange).to_openmm())
    simulation.context.computeVirtualSites()
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(openmm.unit.kelvin * 300)
    simulation.step(1000)

    interchange.positions = simulation.context.getState(
        getPositions=True
    ).getPositions()

    return interchange._visualize_nglview(include_virtual_sites=True)

The first parameter is of type `BondCharge`, which places a virtual site along the axis of a bond. This was originally intended for use with halogens to better model sigma holes. In this case, a  virtual site is added $1.45 \AA$ "outside" a carbon-chlorine bond.

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter({"name": "sigma_hole"})[
    0
].to_dict()

In [ ]:
run("CCCCCl")

Next is a parameter using `MonovalentLonePair`. In this case, a virtual site is added near the oxygen in a carbonyl. It is placed at an angle (`inPlaneAngle`) formed by the oxygen and carbon atoms of the carbonyl and in the plane defined by those atoms and the alpha carbon. If the parameter `outOfPlaneAngle` were non-zero, it would be placed at an angle out of that plane.

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter(
    {"name": "planar_carbonyl"}
)[0].to_dict()

In [ ]:
run("COC1=C(C=CC(=C1)C=O)O")

The next parameter completes a so-called four-site water model like TIP4P or OPC. A single virtual site is placed near the oxygen in the plane of the three atoms. This is implemented with the type `DivalentLonePair`, though it is possible to also implement it with `MonovalentLonePair`.

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter({"name": "4_site_water"})[
    0
].to_dict()

In [ ]:
run("O")

The next parameter uses `TrivalentLonePair` to model the lone pair of a trivalent nitrogen. It is written to match only ammonia - the other capturing atoms are all hydrogens - but a SMIRKS pattern could be written to match trivalent nitrogens in other chemistries. A virtual site is placed $5 \AA$ away from the nitrogen atom, opposite a plane defined by the three hydrogen atoms. (You may need to rotate the molecule, using your cursor, to see the virtual site).

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter(
    {"name": "trivalent_nitrogen"}
)[0].to_dict()

In [ ]:
run("N")

In [ ]:
# run("C1=CC(=CC2=C1O[C@H](CN[S](=O)(=O)N)CO2)Cl")

In [ ]:
Molecule.from_smiles(
    "C1=CC=CC=C1CC(C([O-])=O)N[S](=O)(=O)N",
    allow_undefined_stereo=True,
)

In [ ]:
run("CC=O")